# JSON

## Autores

| Nome | nUSP |
| :--- | :--- |
| Guilherme de Abreu Barreto | 12543033 |
| Lucas Eduardo Gulka Pulcinelli | 12547336 |
| Vinicio Yusuke Hayashibara | 13642797 |

## Enunciado

> Atualize a tabela Premiados, de maneira que o documento associado a cada Premiado possa indicar, além da categoria do Prêmio, também a
especialidade de cada um, quando disponível. Use a tabela `Specialties.csv` disponibilizada no Tidia, em formato `.csv`, como a fonte de dados para essa atualização.
> 
> Suponha que essa informação deve ser armazenada fora do documento Json. Faça atualização correspondente.

## Resolução

Nossa estratégia para encontrar e alterar as entradas da Tabela "Premiados" baseia-se no seguinte procedimento:

1. Acessa-se o documento JSON na tabela "Premiados" e dela se extrai o id, o nome e sobrenome de todos os Laureados.

2. Lê-se e o CSV e constrói-se um padrão regex que permite parear os nomes completos dos Laureados com os nomes dos mesmos descritos por iniciais. A partir deste se obtêm uma paridade de ids com especialidades.

3. Acessa-se novamente o DB para atualizar a coluna "_specialty_" nas entradas de id correspondente.

Foi fator motivante na escolha desta abordagem, com dois acessos ao banco de dados, a ausência de uma normalização consistente nos nomes encontrados no arquivo `Specialties.csv`; com variadas combinações de nomes completos e o uso de iniciais. Por isso julgamos por bem fazer um processamendo dos dados com o uso de Python entre os acessos. Ao final, com a estratégia adotada, fomos capazes de localizar e realizar **35** alterações no banco de dados.

A seguir descrevemos todas as etapas de nossa resolução, incluindo as etapas preliminares de criação e população do banco de dados.

### Configuração

- DEFAULT_DATABASE e NOBEL_DATABASE: Respectivamente, o nome do database padrão e o nome do database onde serão carregadas as informações. Atente-se se este não corresponde ao nome de um database preexistente ou que esteja sendo acessado, pois este será então sobrescrito.

- USER e PASSWORD: Informações de autententicação válidas e com privilégios para a criação de bancos de dados no servidor.

- HOST e PORT: A URL e porta para realização do acesso ao servidor.

In [1]:
DEFAULT_DATABASE = "postgres"
NOBEL_DATABASE = "nobel"
USER = "postgres"
PASSWORD = "postgres"
HOST = "localhost"
PORT = 5432
URI = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}/"

In [2]:
### Carregamento das dependências

In [3]:
import json
import re
import csv
from sqlalchemy import (
    Column,
    Integer,
    String,
    create_engine,
    func,
    select,
    update,
    values,
    text
)
from sqlalchemy.orm import (
    Mapped,
    Session,
    declarative_base,
    sessionmaker,
    mapped_column as column,
)
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.sql.expression import bindparam
from typing import final

In [4]:
### (Re)ciranção do banco de dados "nobel"

In [5]:
engine = create_engine(URI + DEFAULT_DATABASE, echo=True)

with engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
    terminate_sql = text(f"""
        SELECT pg_terminate_backend(pid)
        FROM pg_stat_activity
        WHERE datname = '{NOBEL_DATABASE}';
    """)
    try:
        conn.execute(terminate_sql)
    except ProgrammingError as e:
        print(f"Could not terminate connections (this is often normal): {e}")
    conn.execute(text(f"DROP DATABASE IF EXISTS {NOBEL_DATABASE};"))
    conn.execute(text(f"CREATE DATABASE {NOBEL_DATABASE};")) 

2025-10-04 18:25:03,091 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-10-04 18:25:03,092 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-04 18:25:03,094 INFO sqlalchemy.engine.Engine select current_schema()
2025-10-04 18:25:03,095 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-04 18:25:03,098 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-10-04 18:25:03,099 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-04 18:25:03,102 INFO sqlalchemy.engine.Engine BEGIN (implicit; DBAPI should not BEGIN due to autocommit mode)
2025-10-04 18:25:03,103 INFO sqlalchemy.engine.Engine 
        SELECT pg_terminate_backend(pid)
        FROM pg_stat_activity
        WHERE datname = 'nobel';
    
2025-10-04 18:25:03,104 INFO sqlalchemy.engine.Engine [generated in 0.00253s] {}
2025-10-04 18:25:03,108 INFO sqlalchemy.engine.Engine DROP DATABASE IF EXISTS nobel;
2025-10-04 18:25:03,109 INFO sqlalchemy.engine.Engine [generated in 0.00098s] {}
2025-10-04 18:25:03,142

In [6]:
### Alteração do acesso ao banco de dados padrão para o banco de dados "nobel"

In [7]:
engine = create_engine(URI + NOBEL_DATABASE, echo=True)
Session = sessionmaker(bind=engine)
Base = declarative_base()

### Criação das tabelas

A seguir são definidas todas as tabelas a serem criadas no banco de dados _nobel_. Como se vê, todas as tabelas são baseadas em um mesmo esquema de tabela bastante simples, que contém somente um id serial como chave primária e uma coluna para armazenamendo de uma documento no formato JSONB. Apenas a tabela "Premiados" varia um pouco deste molde ao acrescentar a tabela "specialty", a qual popularemos com os dados extraído do arquivo CSV de referência.

In [8]:
class DocumentTable(Base):
    __abstract__: bool = True
    id: Mapped[int] = column(primary_key=True, autoincrement=True)
    document: Mapped[dict] = column(JSONB)


@final
class Premiado(DocumentTable):
    __tablename__ = "Premiados"
    specialty: Mapped[str | None]


@final
class Premio(DocumentTable):
    __tablename__ = "Premios"


@final
class Pais(DocumentTable):
    __tablename__ = "Paises"

Base.metadata.create_all(engine)

2025-10-04 18:25:03,243 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-10-04 18:25:03,243 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-04 18:25:03,245 INFO sqlalchemy.engine.Engine select current_schema()
2025-10-04 18:25:03,246 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-04 18:25:03,248 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-10-04 18:25:03,249 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-04 18:25:03,251 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-04 18:25:03,258 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

### População das tabelas

Em seguida populamos as tabelas com os dados encontrados nos arquivos JSON de referência.

#### Importante

Note como os arquivos de referência utilziados na execução desta notebook encontram-se todos presentes em uma pasta `datasets` localizada no mesmo diretório que o notebook sendo executado.

In [9]:
entries = []

for filename, key, Table in zip(
    ["Laureate", "Country", "prize"],
    ["laureates", "countries", "prizes"],
    [Premiado, Pais, Premio]
):
    with open(f'datasets/{filename}.json', 'r', encoding='utf-8') as file:
        items = json.load(file)[key]
        entries += [Table(document=item) for item in items]

with Session() as session:
    session.add_all(entries)
    session.commit()

2025-10-04 18:25:03,372 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-04 18:25:03,377 INFO sqlalchemy.engine.Engine INSERT INTO "Paises" (document) SELECT p0::JSONB FROM (VALUES (%(document__0)s::JSONB, 0), (%(document__1)s::JSONB, 1), (%(document__2)s::JSONB, 2), (%(document__3)s::JSONB, 3), (%(document__4)s::JSONB, 4), (%(document__5)s::JSONB, 5), (%(document__6 ... 4105 characters truncated ... , 136)) AS imp_sen(p0, sen_counter) ORDER BY sen_counter RETURNING "Paises".id, "Paises".id AS id__1
2025-10-04 18:25:03,378 INFO sqlalchemy.engine.Engine [generated in 0.00099s (insertmanyvalues) 1/1 (ordered)] {'document__0': '{"name": "Algeria", "code": "DZ"}', 'document__1': '{"name": "Argentina", "code": "AR"}', 'document__2': '{"name": "Australia", "code": "AU"}', 'document__3': '{"name": "Austria", "code": "AT"}', 'document__4': '{"name": "Austria-Hungary"}', 'document__5': '{"name": "Austrian Empire"}', 'document__6': '{"name": "Azerbaijan", "code": "AZ"}', 'document__7': '{"

### Passo 1: Busca dos Laureados

Acessa-se o documento JSON na tabela "Premiados" e dela se extrai o id, o nome e sobrenome de todos os Laureados.

In [10]:
with Session() as session:
    stmt = select(
        Premiado.id,
        Premiado.document['firstname'].label('firstname'),
        Premiado.document['surname'].label('surname')
    )
    result = session.execute(stmt)

laureates = set(
    (row.id, f'{row.firstname} {row.surname}')
    for row in result
)

2025-10-04 18:25:03,570 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-04 18:25:03,575 INFO sqlalchemy.engine.Engine SELECT "Premiados".id, "Premiados".document -> %(document_1)s AS firstname, "Premiados".document -> %(document_2)s AS surname 
FROM "Premiados"
2025-10-04 18:25:03,576 INFO sqlalchemy.engine.Engine [generated in 0.00130s] {'document_1': 'firstname', 'document_2': 'surname'}
2025-10-04 18:25:03,581 INFO sqlalchemy.engine.Engine ROLLBACK


### Passo 2: Pareamento de especialidades com ids utilizando regex

Lê-se o arquivo CSV e constrói-se um padrão regex que permite parear os nomes completos dos Laureados com os nomes dos mesmos descritos por iniciais. A partir deste se obtêm uma paridade de ids com especialidades.

O padrão regex empregado é tal que, na ocorrência de iniciais da forma `A.`, onde `A` é qualuqer inicial, o caractere `.` seja subtituído por um padrão da forma `\w+\s*`. Em uma busca insensível a caia-alta, este padrão encontra a qualquer nome ou sobrenome iniciado por `A` ou `a` seguido de zero ou mais espaços em branco.

In [11]:
def build_initials_regex(name: str) -> re.Pattern:
    pattern = name.replace('.', '\\w+\\s*')
    return re.compile(rf'^{pattern}$', re.IGNORECASE)

# --- Step 2: Prepare update_params ---
update_params = []
with open(f'datasets/Specialties.csv', 'r', encoding='utf-8') as file:
    for row in list(csv.DictReader(file)):
        for laureate in laureates:
            pattern = build_initials_regex(row['Laureate'])
            if re.match(pattern, laureate[1]):
                update_params.append({
                    "p_id": laureate[0],
                    "p_specialty": row['Specialty']
                })
                laureates.remove(laureate)
                break

### Passo 3: Atualização da tabela `Premiados`

Acessa-se novamente o DB para atualizar a coluna "_specialty_" nas entradas de id correspondente.

In [12]:
with Session() as session:
    for params in update_params:
        stmt = (
            update(Premiado)
            .where(Premiado.id == bindparam('p_id'))
            .values(
                specialty=bindparam('p_specialty')
            )
            .returning(
                Premiado.document['firstname'].label('firstname'),
                Premiado.document['surname'].label('surname'),
                Premiado.specialty
            )
        )
        result = session.execute(stmt, params).first()
        print(f"✅ Updated: Name={result.firstname}, Surname={result.surname}, Specialty={result.specialty}")
    session.commit()
    print(f"✅ Updated {len(update_params)} records.")

2025-10-04 18:25:03,828 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-04 18:25:03,834 INFO sqlalchemy.engine.Engine UPDATE "Premiados" SET specialty=%(p_specialty)s WHERE "Premiados".id = %(p_id)s RETURNING "Premiados".document -> %(document_1)s AS firstname, "Premiados".document -> %(document_2)s AS surname, "Premiados".specialty
2025-10-04 18:25:03,836 INFO sqlalchemy.engine.Engine [generated in 0.00168s] {'p_specialty': 'Cristallography', 'p_id': 747, 'document_1': 'firstname', 'document_2': 'surname'}
✅ Updated: Name=Peter, Surname=Agre, Specialty=Cristallography
2025-10-04 18:25:03,842 INFO sqlalchemy.engine.Engine UPDATE "Premiados" SET specialty=%(p_specialty)s WHERE "Premiados".id = %(p_id)s RETURNING "Premiados".document -> %(document_1)s AS firstname, "Premiados".document -> %(document_2)s AS surname, "Premiados".specialty
2025-10-04 18:25:03,844 INFO sqlalchemy.engine.Engine [cached since 0.009859s ago] {'p_specialty': 'Cristallography', 'p_id': 237, 'document_1': '

## 